In [1]:
#install the package newspaper 3k
!pip install newspaper3k

In [2]:
#import libraries
from newspaper import Article
import random
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import numpy as np
import warnings
import re

In [3]:
#Ignore warnings 
warnings.filterwarnings("ignore")

In [4]:
#Download the packages from NLTK
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\elgha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\elgha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
#Get the Article URL
article=Article("https://www.mayoclinic.org/diseases-conditions/depression/symptoms-causes/syc-20356007")
article.download()
article.parse()
article.nlp()
corpus=article.text

In [6]:
text=corpus
print(text)

Overview

Depression is a mood disorder that causes a persistent feeling of sadness and loss of interest. Also called major depressive disorder or clinical depression, it affects how you feel, think and behave and can lead to a variety of emotional and physical problems. You may have trouble doing normal day-to-day activities, and sometimes you may feel as if life isn't worth living.

More than just a bout of the blues, depression isn't a weakness and you can't simply "snap out" of it. Depression may require long-term treatment. But don't get discouraged. Most people with depression feel better with medication, psychotherapy or both.

Depression care at Mayo Clinic

Symptoms

Although depression may occur only once during your life, people typically have multiple episodes. During these episodes, symptoms occur most of the day, nearly every day and may include:

Feelings of sadness, tearfulness, emptiness or hopelessness

Angry outbursts, irritability or frustration, even over small mat

In [7]:
#Sentece Tokenization

sent_tokens=nltk.sent_tokenize(text)
print(sent_tokens)

['Overview\n\nDepression is a mood disorder that causes a persistent feeling of sadness and loss of interest.', 'Also called major depressive disorder or clinical depression, it affects how you feel, think and behave and can lead to a variety of emotional and physical problems.', "You may have trouble doing normal day-to-day activities, and sometimes you may feel as if life isn't worth living.", 'More than just a bout of the blues, depression isn\'t a weakness and you can\'t simply "snap out" of it.', 'Depression may require long-term treatment.', "But don't get discouraged.", 'Most people with depression feel better with medication, psychotherapy or both.', 'Depression care at Mayo Clinic\n\nSymptoms\n\nAlthough depression may occur only once during your life, people typically have multiple episodes.', 'During these episodes, symptoms occur most of the day, nearly every day and may include:\n\nFeelings of sadness, tearfulness, emptiness or hopelessness\n\nAngry outbursts, irritability

In [8]:
#Remove Punctuation
remove_punct_dict=dict( (ord(punct),None)for punct in string.punctuation)

In [9]:
print(remove_punct_dict)

{33: None, 34: None, 35: None, 36: None, 37: None, 38: None, 39: None, 40: None, 41: None, 42: None, 43: None, 44: None, 45: None, 46: None, 47: None, 58: None, 59: None, 60: None, 61: None, 62: None, 63: None, 64: None, 91: None, 92: None, 93: None, 94: None, 95: None, 96: None, 123: None, 124: None, 125: None, 126: None}


In [10]:
#lemmatization
def lemmatizing(text):
    return nltk.word_tokenize(text.lower().translate(remove_punct_dict))
print(lemmatizing(text))

['overview', 'depression', 'is', 'a', 'mood', 'disorder', 'that', 'causes', 'a', 'persistent', 'feeling', 'of', 'sadness', 'and', 'loss', 'of', 'interest', 'also', 'called', 'major', 'depressive', 'disorder', 'or', 'clinical', 'depression', 'it', 'affects', 'how', 'you', 'feel', 'think', 'and', 'behave', 'and', 'can', 'lead', 'to', 'a', 'variety', 'of', 'emotional', 'and', 'physical', 'problems', 'you', 'may', 'have', 'trouble', 'doing', 'normal', 'daytoday', 'activities', 'and', 'sometimes', 'you', 'may', 'feel', 'as', 'if', 'life', 'isnt', 'worth', 'living', 'more', 'than', 'just', 'a', 'bout', 'of', 'the', 'blues', 'depression', 'isnt', 'a', 'weakness', 'and', 'you', 'cant', 'simply', 'snap', 'out', 'of', 'it', 'depression', 'may', 'require', 'longterm', 'treatment', 'but', 'dont', 'get', 'discouraged', 'most', 'people', 'with', 'depression', 'feel', 'better', 'with', 'medication', 'psychotherapy', 'or', 'both', 'depression', 'care', 'at', 'mayo', 'clinic', 'symptoms', 'although', '

In [11]:
#Keyword Matching and Greetings
greeting_inputs=['hello','Hola','Hi','whats up','hey']
greeting_response=['hi','hello','howdy','hola','hey',"what's good"]
def greeting(sentence):
    for word in sentence.split():
        if word.lower()in greeting_inputs:
            return random.choice(greeting_response)

In [12]:
#Generate Response
def response (user_response): 
    
    #user response/ query
    #user_response='What is depression?'
    user_response=user_response.lower()
    #print(user_response)


    #set the chatbot response to an empty string
    robo_response=''

    #append the user response to the sentence list
    sent_tokens.append(user_response)
    #print('Sentence tokens: ',sent_tokens)

    #Tfidf vectorizer 
    TfidfVec=TfidfVectorizer(tokenizer=lemmatizing,stop_words='english')

    #convert the text to a matrix of Tfidf features
    tfidf=TfidfVec.fit_transform(sent_tokens)
    #print("Tfidf features: ",tfidf)

    #Get the measure of similarity(similarity score)
    vals=cosine_similarity(tfidf[-1],tfidf)
    #print("Similarity scores: " ,vals)

    #Get the index of the most similar sentence to the user response
    idx=vals.argsort()[0][-2]

    #Reduce the dimensionality of vals
    flat=vals.flatten()

    #sort the list in ascending order
    flat.sort()

    #Get the most similar score to the user response
    score=flat[-2]
    #print(score)

    #if the variable score is 0 then there is no text similar to the user response
    if(score==0):
        robo_response=robo_response+"I don't understand!"
    else:
        robo_response=robo_response+sent_tokens[idx]

    #print("Bot Response: ",robo_response)   
    
    #remove the user response
    sent_tokens.remove(user_response)
    
    return robo_response

In [13]:
#creat a loop to chat with the chatbot
flag=True
print("DocBot: I will answer your queries about Depression disease. if you want to exit just type bye")
while(flag==True):
    user_response=input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks'or user_response=='thank you'):
            flag=False
            print("DocBot: you are welcome!")
        else: 
            if(greeting(user_response)!=None):
                print("DocBot: "+greeting(user_response))
            else:
                print("DocBot: "+response(user_response))
                
    else:
        flag=False
        print("DocBot: Chat with you later ! ")

DocBot: I will answer your queries about Depression disease. if you want to exit just type bye
hey
DocBot: what's good
what is depression?
DocBot: Depression symptoms in older adults

Depression is not a normal part of growing older, and it should never be taken lightly.
what are the symptoms of depression?
DocBot: Depression symptoms in children and teens

Common signs and symptoms of depression in children and teenagers are similar to those of adults, but there can be some differences.
thanks
DocBot: you are welcome!
